In [16]:
import pandas as pd
import numpy as np
import plotly.express as px

In [17]:
%config Completer.use_jedi = False

In [18]:
covid_data = pd.read_csv("./public/data/COVID-19_aantallen_gemeente_cumulatief.csv", delimiter=";")
covid_data

,Date_of_report,Municipality_code,Municipality_name,Province,Total_reported,Hospital_admission,Deceased
0,2020-03-13 10:00:00,GM0003,Appingedam,Groningen,0,0,0
1,2020-03-13 10:00:00,GM0010,Delfzijl,Groningen,0,0,0
2,2020-03-13 10:00:00,GM0014,Groningen,Groningen,3,0,0
3,2020-03-13 10:00:00,GM0024,Loppersum,Groningen,0,0,0
4,2020-03-13 10:00:00,GM0034,Almere,Flevoland,1,1,0
...,...,...,...,...,...,...,...
216747,2021-10-26 10:00:00,NaN,NaN,Noord-Holland,1620,24,2
216748,2021-10-26 10:00:00,NaN,NaN,Zuid-Holland,3395,65,15
216749,2021-10-26 10:00:00,NaN,NaN,Zeeland,108,5,1
216750,2021-10-26 10:00:00,NaN,NaN,Noord-Brabant,1635,38,8


# Clean and drop all NaNs

In [19]:
covid_data_cleaned = covid_data.dropna().copy()
covid_data_cleaned

,Date_of_report,Municipality_code,Municipality_name,Province,Total_reported,Hospital_admission,Deceased
0,2020-03-13 10:00:00,GM0003,Appingedam,Groningen,0,0,0
1,2020-03-13 10:00:00,GM0010,Delfzijl,Groningen,0,0,0
2,2020-03-13 10:00:00,GM0014,Groningen,Groningen,3,0,0
3,2020-03-13 10:00:00,GM0024,Loppersum,Groningen,0,0,0
4,2020-03-13 10:00:00,GM0034,Almere,Flevoland,1,1,0
...,...,...,...,...,...,...,...
216735,2021-10-26 10:00:00,GM1966,Het Hogeland,Groningen,4633,39,20
216736,2021-10-26 10:00:00,GM1969,Westerkwartier,Groningen,4477,39,26
216737,2021-10-26 10:00:00,GM1970,Noardeast-FryslÃ¢n,FryslÃ¢n,4693,63,40
216738,2021-10-26 10:00:00,GM1978,Molenlanden,Zuid-Holland,7014,57,36


In [20]:
covid_data_cleaned.sort_values(["Date_of_report", "Municipality_code"], inplace=True)

In [21]:
covid_data_cleaned["Daily_reported"] = covid_data_cleaned.groupby("Municipality_code")["Total_reported"].transform(lambda x: x.diff())
covid_data_cleaned.dropna(inplace=True)

In [22]:
covid_data_cleaned["Daily_reported_moving_average"] =\
    covid_data_cleaned.groupby("Municipality_code")["Daily_reported"].transform(
        lambda x: x.rolling(
            window=14,
            min_periods=1,
            center=True,
            
        ).mean()
)
covid_data_cleaned["Daily_reported_moving_average"] = covid_data_cleaned["Daily_reported_moving_average"].round(2)

In [23]:
covid_data_cleaned["Date_of_report"] = pd.to_datetime(covid_data_cleaned["Date_of_report"])

In [24]:
covid_data_cleaned[covid_data_cleaned["Municipality_name"] == "Eindhoven"]

,Date_of_report,Municipality_code,Municipality_name,Province,Total_reported,Hospital_admission,Deceased,Daily_reported,Daily_reported_moving_average
579,2020-03-14 10:00:00,GM0772,Eindhoven,Noord-Brabant,13,0,0,1.0,4.43
946,2020-03-15 10:00:00,GM0772,Eindhoven,Noord-Brabant,21,0,0,8.0,3.00
1313,2020-03-16 10:00:00,GM0772,Eindhoven,Noord-Brabant,21,0,0,0.0,3.89
1680,2020-03-17 10:00:00,GM0772,Eindhoven,Noord-Brabant,22,0,0,1.0,4.50
2047,2020-03-18 10:00:00,GM0772,Eindhoven,Noord-Brabant,23,1,0,1.0,5.27
...,...,...,...,...,...,...,...,...,...
215141,2021-10-22 10:00:00,GM0772,Eindhoven,Noord-Brabant,27854,471,361,15.0,36.42
215505,2021-10-23 10:00:00,GM0772,Eindhoven,Noord-Brabant,27918,471,361,64.0,38.45
215869,2021-10-24 10:00:00,GM0772,Eindhoven,Noord-Brabant,27955,471,361,37.0,39.70
216233,2021-10-25 10:00:00,GM0772,Eindhoven,Noord-Brabant,28026,471,361,71.0,40.22


In [25]:
df_eindhoven = covid_data_cleaned[covid_data_cleaned["Municipality_name"] == "Eindhoven"]
df_eindhoven.head()

,Date_of_report,Municipality_code,Municipality_name,Province,Total_reported,Hospital_admission,Deceased,Daily_reported,Daily_reported_moving_average
579,2020-03-14 10:00:00,GM0772,Eindhoven,Noord-Brabant,13,0,0,1.0,4.43
946,2020-03-15 10:00:00,GM0772,Eindhoven,Noord-Brabant,21,0,0,8.0,3.00
1313,2020-03-16 10:00:00,GM0772,Eindhoven,Noord-Brabant,21,0,0,0.0,3.89
1680,2020-03-17 10:00:00,GM0772,Eindhoven,Noord-Brabant,22,0,0,1.0,4.50
2047,2020-03-18 10:00:00,GM0772,Eindhoven,Noord-Brabant,23,1,0,1.0,5.27


In [26]:
px.line(
    covid_data_cleaned[covid_data_cleaned["Municipality_name"] == "Utrecht"], 
    x="Date_of_report",
    y="Daily_reported_moving_average",
)

In [27]:
px.line(
    covid_data_cleaned[covid_data_cleaned["Municipality_name"] == "Utrecht"], 
    x="Date_of_report",
    y="Daily_reported",
)

# Write to CSV

In [28]:
covid_data_cleaned.to_csv("./public/data/COVID-19-NL-Municipality-Wise.csv", index=False)

# Check missing municipalities

In [30]:
population_data = pd.read_csv("./public/data/NL_Population_Latest.csv")
population_data

,ID,Sex,Regions,Periods,PopulationOn1January_1,PopulationOn31December_20
0,6239,T001038,GM0003,2019JJ00,11721.0,11642.0
1,7858,T001038,GM0005,2018JJ00,10475.0,10493.0
2,8637,T001038,GM0007,2017JJ00,8919.0,8816.0
3,11758,T001038,GM0009,2018JJ00,7292.0,7289.0
4,15239,T001038,GM0010,2019JJ00,24716.0,24678.0
...,...,...,...,...,...,...
404,29879,T001038,GM1966,2019JJ00,47888.0,47801.0
405,67979,T001038,GM1969,2019JJ00,63031.0,63329.0
406,44819,T001038,GM1970,2019JJ00,45181.0,45228.0
407,41639,T001038,GM1978,2019JJ00,43858.0,43909.0


In [31]:
all_municipalities_covid_data = set(covid_data_cleaned["Municipality_code"])
all_municipalities_population_data = set(population_data["Regions"])

In [32]:
all_municipalities_covid_data - all_municipalities_population_data

{'GM1979'}

## Municipalities not present in population data

In [33]:
missing_df = covid_data_cleaned[covid_data_cleaned["Municipality_code"].isin(
    list(all_municipalities_covid_data - all_municipalities_population_data))
][["Municipality_name", "Province"]]
list(missing_df.groupby(["Municipality_name", "Province"]).groups.keys())

[('Eemsdelta', 'Groningen')]

## Municipalities not present in COVID data

In [34]:
missing_df = covid_data_cleaned[covid_data_cleaned["Municipality_code"].isin(
    list(all_municipalities_population_data - all_municipalities_covid_data))
][["Municipality_name", "Province"]]
list(missing_df.groupby(["Municipality_name", "Province"]).groups.keys())

[]